In [10]:
# impert pytorch
import numpy as np
import pandas as pd

filename = 'Final_Data.csv'
data = pd.read_csv(filename)

data.columns

Index(['Season', 'Home', 'Away', 'FTR', 'HomeTeam_WinPercentage',
       'HomeTeam_DrawPercentage', 'AwayTeam_WinPercentage',
       'AwayTeam_DrawPercentage', 'HomeTeam_Streak', 'AwayTeam_Streak',
       'HomeTeam_AvgGoal', 'AwayTeam_AvgGoal', 'HomeTeam_AvgGoalDiff',
       'AwayTeam_AvgGoalDiff', 'HomeTeam_AvgShotOnTarget',
       'AwayTeam_AvgShotOnTarget', 'HomeTeam_AvgShotBlock',
       'AwayTeam_AvgShotBlock', 'HomeTeam_AvgCorners', 'AwayTeam_AvgCorners',
       'HomeTeam_AvgRedCards', 'AwayTeam_AvgRedCards',
       'HomeTeam_AvgYellowCards', 'AwayTeam_AvgYellowCards',
       'HomeTeam_H2H_Wins', 'AwayTeam_H2H_Wins', 'HomeTeam_Last_Season',
       'AwayTeam_Last_Season', 'Time_Weight', 'Text'],
      dtype='object')

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

df = data

# Label Encoding
label_encoder = LabelEncoder()
df['FTR'] = label_encoder.fit_transform(df['FTR'])

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

# Calculate Time Weights
alpha = 0.6
weights = alpha ** (1 - df['Time_Weight'])

# Prepare Features and Target
X = df.drop(columns=['Season', 'Home', 'Away', 'FTR', 'Time_Weight', 'Text'])
y = df['FTR']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost Classifier with Default Parameters
model = xgb.XGBClassifier(objective='multi:softmax',
                          num_class=3,
                          n_estimators=100,
                          learning_rate=0.1,
                          max_depth=5,
                          random_state=42)

# Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
mean_cv_accuracy = np.mean(cross_val_scores) * 100
print(f'Mean Cross-Validation Accuracy: {mean_cv_accuracy:.2f}%')

# Model Fitting
model.fit(X_train, y_train, sample_weight=weights.loc[y_train.index].values)

# Prediction
y_pred = model.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred) * 100
print(f'Accuracy: {accuracy:.2f}%')
print('Classification Report:')
print(classification_report(y_test, y_pred))



{0: 0, 1: 1, 2: 2}
Mean Cross-Validation Accuracy: 53.55%
Accuracy: 60.82%
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.59      0.60        80
           1       0.46      0.17      0.24        66
           2       0.63      0.86      0.73       122

    accuracy                           0.61       268
   macro avg       0.57      0.54      0.52       268
weighted avg       0.58      0.61      0.57       268

